# CF Analyzer

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

from joblib import load
import dice_ml

In [2]:
def adjust_data_types(df):
    for col in df.columns:
        unique_vals = df[col].unique()
        if df[col].dtypes == 'object':
            None
#         elif (set(unique_vals) == {0, 1}) or col == 'userId':
#             df[col] = df[col].astype('int64')
#         else:
#             df[col] = df[col].astype('float64')
        elif (set(unique_vals) != {0, 1}):
            df[col] = df[col].astype('float64')
        else:
            df[col] = df[col].astype('int64')
    return df

In [3]:
t01 = pd.read_csv('inputs/Example5.csv')

In [4]:
df_n = t01.copy()
adjust_data_types(df_n)
X = df_n

In [5]:
exp = load('dice_model/dice_exp.joblib')

In [8]:
# Generate counterfactual examples
features_to_vary = [
        'food',
        'hygiene', 'houseCleaning', 'fare', 'parking', 'gasoline', 
        'water', 'electricity', 'emergency', 'tuition', 'internet',
        'allowance', 'uniform', 'otherEducation', 'medicine', 'rent', 'repair',
        'cinema', 'dineOut', 'leisure', 'personalCare', 'clothing',
        'mobileLoad',
        'smoking', 'alcohol', 'gambling', 'smallLottery', 'otherVices',
        'savings'
    ]

# jk = 197

query_instance = X
permissible_range = {feature: [0, query_instance[feature].values[0]] for feature in features_to_vary}

MAX_SAVINGS = 0
if query_instance['savings'].values[0] == 0:
    MAX_SAVINGS = 0.3 * query_instance['basicMonthlySalary'].values[0]
else:
    MAX_SAVINGS = 0.3 * query_instance['basicMonthlySalary'].values[0]

permissible_range['savings'] = [query_instance['savings'].values[0], MAX_SAVINGS]

dice_exp = exp.generate_counterfactuals(
    query_instance,
    total_CFs=5,
    desired_class="opposite",
    # verbose=True,
    # features_to_vary=features_to_vary,
    permitted_range=permissible_range,
    # proximity_weight=1,
    # diversity_weight=0.5
)
# Visualize counterfactual explanation
dice_exp.visualize_as_dataframe(
    # display_sparse_df=False,
                                show_only_changes=True)

100%|██████████| 1/1 [00:00<00:00,  1.04it/s]

Query instance (original outcome : 0)


,age,basicMonthlySalary,preferredNetDisposableIncomeId,workingFamilyCount,residentsCount,monthlyFamilyIncome,food,hygiene,houseCleaning,fare,parking,gasoline,tuition,allowance,uniform,otherEducation,emergency,medicine,water,electricity,rent,repair,cinema,dineOut,leisure,personalCare,clothing,mobileLoad,internet,vehicleLoan,informalLenders,companyLoan,privateLoans,governmentLoans,smoking,alcohol,gambling,smallLottery,otherVices,savings,loanOthers,payInsurance,loanSSS,payFamilySupport,loanPagIbig,loanGSIS,loanPersonal,houseHasPensioner,houseHasPrivateEmployee,houseHasBusiness,houseHasFreelancer,houseHasGovtEmployee,houseHasOFW,houseOnlyFamily,houseExtendedFamily,daysFromDateEntryStart,monthlyUtilityBills,monthlyVices,monthlyExpenses,monthlySoloNetIncome,positiveMonthlySoloNetIncome,monthlyFamilyNetIncome,positiveMonthlyFamilyNetIncome,monthlySoloNetIncomeWithSavings,positiveMonthlySoloNetIncomeWithSavings,monthlyFamilyNetIncomeWithSavings,positiveMonthlyFamilyNetIncomeWithSavings,monthlyFamilyIncome - basicMonthlySalary,positive monthlyFamilyIncome - basicMonthlySalary,basicMonthlySalary - monthlyExpenses,positive basicMonthlySalary - monthlyExpenses,monthlyFamilyIncome - monthlyExpenses,positive monthlyFamilyIncome - monthlyExpenses,basicMonthlySalary / monthlyFamilyIncome,monthlyExpenses / monthlyFamilyIncome,monthlyVices / monthlyFamilyIncome,gender_MALE,province_AKLAN,province_ALBAY,province_BATAAN,province_BATANGAS,province_BULACAN,province_CAGAYAN,province_CAMARINES SUR,province_CAVITE,province_CEBU,province_COTABATO,province_DAVAO DEL SUR,province_EASTERN SAMAR,province_FOREIGN COUNTRY,province_ILOCOS NORTE,province_ILOILO,province_ISABELA,province_LAGUNA,province_LEYTE,province_MARINDUQUE,province_METRO MANILA,province_MISAMIS OCCIDENTAL,province_NEGROS OCCIDENTAL,province_NUEVA ECIJA,province_ORIENTAL MINDORO,province_PANGASINAN,province_QUEZON,province_QUIRINO,province_RIZAL,province_ROMBLON,province_SOUTH COTABATO,province_TARLAC,province_UNKNOWN,province_ZAMBOANGA DEL NORTE,job_ARMED FORCES,job_ASSOCIATE PROFESSIONAL,job_CLERICAL SUPPORT,job_CRAFT AND TRADE,job_ELEMENTARY OCCUPATION,job_MACHINE OPERATOR,job_MANAGER,job_NONE,job_PROFESIONAL,job_SERVICE AND SALES,job_UNKNOWN,home_ownership_class
0,23.0,21000.0,2.0,3.0,5.0,95000.0,2500.0,1000.0,500.0,1500.0,0.0,0.0,0.0,500.0,0.0,0.0,2000.0,500.0,0.0,0.0,0.0,500.0,0.0,500.0,500.0,250.0,200.0,0.0,0.0,0.0,0.0,0.0,500.0,250.0,0.0,0.0,0.0,0.0,0.0,5000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,194.0,0.0,0.0,11200.0,9800.0,1.0,83800.0,1.0,14800.0,1.0,88800.0,1.0,74000.0,1.0,9800.0,1.0,83800.0,1.0,0.221053,0.117895,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0



Diverse Counterfactual set (new outcome: 1)


,age,basicMonthlySalary,preferredNetDisposableIncomeId,workingFamilyCount,residentsCount,monthlyFamilyIncome,food,hygiene,houseCleaning,fare,parking,gasoline,tuition,allowance,uniform,otherEducation,emergency,medicine,water,electricity,rent,repair,cinema,dineOut,leisure,personalCare,clothing,mobileLoad,internet,vehicleLoan,informalLenders,companyLoan,privateLoans,governmentLoans,smoking,alcohol,gambling,smallLottery,otherVices,savings,loanOthers,payInsurance,loanSSS,payFamilySupport,loanPagIbig,loanGSIS,loanPersonal,houseHasPensioner,houseHasPrivateEmployee,houseHasBusiness,houseHasFreelancer,houseHasGovtEmployee,houseHasOFW,houseOnlyFamily,houseExtendedFamily,daysFromDateEntryStart,monthlyUtilityBills,monthlyVices,monthlyExpenses,monthlySoloNetIncome,positiveMonthlySoloNetIncome,monthlyFamilyNetIncome,positiveMonthlyFamilyNetIncome,monthlySoloNetIncomeWithSavings,positiveMonthlySoloNetIncomeWithSavings,monthlyFamilyNetIncomeWithSavings,positiveMonthlyFamilyNetIncomeWithSavings,monthlyFamilyIncome - basicMonthlySalary,positive monthlyFamilyIncome - basicMonthlySalary,basicMonthlySalary - monthlyExpenses,positive basicMonthlySalary - monthlyExpenses,monthlyFamilyIncome - monthlyExpenses,positive monthlyFamilyIncome - monthlyExpenses,basicMonthlySalary / monthlyFamilyIncome,monthlyExpenses / monthlyFamilyIncome,monthlyVices / monthlyFamilyIncome,gender_MALE,province_AKLAN,province_ALBAY,province_BATAAN,province_BATANGAS,province_BULACAN,province_CAGAYAN,province_CAMARINES SUR,province_CAVITE,province_CEBU,province_COTABATO,province_DAVAO DEL SUR,province_EASTERN SAMAR,province_FOREIGN COUNTRY,province_ILOCOS NORTE,province_ILOILO,province_ISABELA,province_LAGUNA,province_LEYTE,province_MARINDUQUE,province_METRO MANILA,province_MISAMIS OCCIDENTAL,province_NEGROS OCCIDENTAL,province_NUEVA ECIJA,province_ORIENTAL MINDORO,province_PANGASINAN,province_QUEZON,province_QUIRINO,province_RIZAL,province_ROMBLON,province_SOUTH COTABATO,province_TARLAC,province_UNKNOWN,province_ZAMBOANGA DEL NORTE,job_ARMED FORCES,job_ASSOCIATE PROFESSIONAL,job_CLERICAL SUPPORT,job_CRAFT AND TRADE,job_ELEMENTARY OCCUPATION,job_MACHINE OPERATOR,job_MANAGER,job_NONE,job_PROFESIONAL,job_SERVICE AND SALES,job_UNKNOWN,home_ownership_class
0,51.0,-,3.0,-,-,-,-,-,-,-,-,-,-,-,-,-,1000.0,-,-,-,-,0.0,-,-,-,-,-,-,-,-,-,-,0.0,0.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1.0,1.0,-,-,-,-,209.0,1.0,5700.0,-,-,-,-,-,-,-,-,-,-,-,1.0,-,-,-,1.0,0.8,0.6,-,-,-,-,-,-,-,-,-,1.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0.0,-,-,-,-,-,-,-,-,-,-,-,-,1.0
0,42.0,-,3.0,2.0,4.0,-,-,-,-,500.0,-,-,-,-,-,-,1500.0,-,-,-,-,0.0,-,-,-,200.0,-,-,-,-,-,-,0.0,0.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1.0,-,-,199.0,1.0,4700.0,-,-,-,-,-,-,-,-,-,-,-,1.0,-,-,-,1.0,0.9,0.6,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0.0,-,-,-,0.0,-,-,-,-,-,-,-,1.0,1.0
0,36.0,-,3.0,4.0,6.0,-,-,-,-,0.0,-,-,-,-,-,-,0.0,-,-,-,-,-,-,-,0.0,0.0,-,-,-,-,1.0,-,0.0,2.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,177.0,1.0,7800.0,-,-,-,-,-,-,-,-,-,-,-,1.0,-,-,-,1.0,0.3,0.3,-,1.0,-,-,-,-,-,-,-,1.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0.0,-,-,-,0.0,1.0,-,-,-,-,-,-,-,1.0
0,39.0,-,3.0,-,7.0,-,-,-,-,-,-,-,-,-,-,-,500.0,-,-,-,-,0.0,-,0.0,0.0,-,-,-,-,-,-,-,0.0,0.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0.0,1.0,201.0,1.0,2100.0,-,-,-,-,-,-,-,-,-,-,-,1.0,-,-,-,1.0,0.7,0.2,-,1.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1.0,-,-,-,-,-,-,-,-,-,-,0.0,-,-,-,0.0,-,-,-,-,1.0,-,-,-,1.0
0,36.0,-,3.0,2.0,3.0,-,-,200.0,-,1000.0,-,-,-,0.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0.0,0.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0.0,1.0,228.0,1.0,6750.0,-,-,-,-,-,-,-,-,-,-,0.0,0.0,-,-,-,1.0,1.0,0.6,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0.0,-,1.0,-,0.0,-,-,-,1.0,-,-,-,-,1.0
